In [9]:
import requests
import time
import warnings

! nest-server start -o -d -p 52425
time.sleep(1.0)
    
try:
    print(requests.get('http://localhost:52425').json())
    print(requests.get('http://127.0.0.1:52425/api/GetKernelStatus', json=["biological_time"]).json())
    RAY = False
except Exception as e:
    
    warnings.warn("Failed to start and deamonize NEST server with error:\n" + 
                  "%s\nSwitching to Ray deamonization!" % str(e))
    
    RAY = True
    # TODO: Find out why we can't deamonize the NEST server...
    # For the moment we need to use Ray to deamonize it, or start it on a terminal.
    # ! nest-server start -o -p 52425

    import os
    import ray

    ray.init()  # _temp_dir="/home/docker/tmp/ray", object_store_memory=100000000


    @ray.remote
    def start_deamonize_nest_server():
        os.system('! nest-server start -o -p 52425')

    # @ray.remote
    # def nest_server_status():
    #     os.system('! nest-server status')

    # @ray.remote
    # def nest_server_log():
    #     os.system('! nest-server log')

    start_deamonize_nest_server.remote()
    # nest_server_status.remote()
    # nest_server_log.remote()
    
    time.sleep(1.0)
    print(requests.get('http://localhost:52425').json())
    print(requests.get('http://127.0.0.1:52425/api/GetKernelStatus', json=["biological_time"]).json())
    

NEST Server is now running at http://127.0.0.1:52425.


/tmp/ipykernel_539/2236785663.py:14: UserWarning: Failed to start and deamonize NEST server with error:
HTTPConnectionPool(host='localhost', port=52425): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff6441af80>: Failed to establish a new connection: [Errno 111] Connection refused'))
Switching to Ray deamonization!
  warnings.warn("Failed to start and deamonize NEST server with error:\n" +
2023-09-20 15:13:12,587	INFO worker.py:1642 -- Started a local Ray instance.


(start_deamonize_nest_server pid=915) NEST Server is now running at http://127.0.0.1:52425.
(start_deamonize_nest_server pid=915) Use CTRL + C to stop this service.
(start_deamonize_nest_server pid=915) -------------------------------------------------
(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915)               -- N E S T --
(start_deamonize_nest_server pid=915)   Copyright (C) 2004 The NEST Initiative
(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915)  Version: 3.5
(start_deamonize_nest_server pid=915)  Built: Sep 20 2023 14:50:00
(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915)  This program is provided AS IS and comes with
(start_deamonize_nest_server pid=915)  NO WARRANTY. See the file LICENSE for details.
(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915)  Problems or suggestions?
(start_deamonize_nest_server pid=915)    Visit https://www.nest-simulator.org
(start_deamoni

In [10]:
from nest_client import NESTClient

print('Running client examples using NEST via NEST Server')

# Load NEST Server client
nestsc = NESTClient(host="127.0.0.1", port=52425)

#
# Use NEST Server API
#
print('\n')
print('Execute script code with NEST Server API')
print('-' * 20)

# Reset kernel
nestsc.ResetKernel()
nestsc.GetKernelStatus("biological_time")

# Create nodes
pg = nestsc.Create("poisson_generator", params={"rate": 6500.})
neurons = nestsc.Create("iaf_psc_alpha", 100)
sr = nestsc.Create("spike_recorder")

# Connect nodes
nestsc.Connect(pg, neurons, syn_spec={'weight': 10.})
nestsc.Connect(neurons[::10], sr)

# Simulate
nestsc.Simulate(1000.0)

# Get events
n_events = nestsc.GetStatus(sr, 'n_events')[0]
print('Number of events:', n_events)


#
# Use NEST Server exec
#
print('\n')
print('Execute script code from file')
print('-' * 20)

n_events = nestsc.from_file('/home/docker/packages/nest-client/examples/NESTClient_script.py', 'n_events')['data']
print('Number of events:', n_events)

Running client examples using NEST via NEST Server


Execute script code with NEST Server API
--------------------
Number of events: 126


Execute script code from file
--------------------
Execute script code of /home/docker/packages/nest-client/examples/NESTClient_script.py
Return variables: n_events
--------------------
# -*- coding: utf-8 -*-
#
# NESTClient_script.py
#
# This file is part of NEST.
#
# Copyright (C) 2004 The NEST Initiative
#
# NEST is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 2 of the License, or
# (at your option) any later version.
#
# NEST is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# al

(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915) Sep 20 15:13:20 SimulationManager::run [Info]: 
(start_deamonize_nest_server pid=915)     Simulation finished.


In [11]:
from tvb_multiscale.tvb_nest.nest_models.server_client.nest_server_client import NESTServerClient 

# Load NEST Server client
nest = NESTServerClient(host='localhost', port=52425)

nest.GetKernelStatus("biological_time")

1000.0

(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915) ========================================
(start_deamonize_nest_server pid=915) 
(start_deamonize_nest_server pid=915) ==> MASTER 0/1695222801.7650070 (route_api_call): call=GetKernelStatus, args=['biological_time'], kwargs={}
(start_deamonize_nest_server pid=915) ==> MASTER 0/1695222801.7651644 (GetKernelStatus): local call, args=['biological_time'], kwargs={}


In [12]:
! curl localhost:52425

/usr/bin/sh: 1: curl: not found


In [13]:
! nest-server stop

NEST Server running at http://127.0.0.1:52425 has stopped.


In [14]:
if RAY: 
    ray.shutdown()